In [ ]:
!pip install openpyxl
!pip install selenium
!pip install xlrd

In [7]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


In [8]:
def obter_elemento(driver, xpath):
    try:
        elem = WebDriverWait(driver, 50).until(EC.presence_of_element_located((By.XPATH, xpath)))
        elem = driver.find_element(By.XPATH, xpath)
        return elem
    except:
        return False

In [3]:
def click(driver, xpath):
    elem = driver.find_element(By.XPATH, xpath)
    elem.click()

In [9]:
def obter_valor(driver, xpath):
    elem = driver.find_element(By.XPATH, xpath)
    return elem.get_attribute("value")

In [11]:
driver = webdriver.Chrome()
driver.get("https://inova.coop.br/radar#cases")
try:
    #botão de aceitar cookies
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='adopt-accept-all-button']"))).click()
finally:
    #botão de fechar barra roxa inferior
    click(driver, "/html/body/app-root/mat-drawer-container/mat-drawer-content/div/div/div/app-innovation-radar-page/app-send-case/div/div[2]/button")

    #enquanto acha a barra 'ver mais'...
    while obter_elemento(driver, "/html/body/app-root/mat-drawer-container/mat-drawer-content/div/div/div/app-innovation-radar-page/app-container/div/div/app-paginated-content/div/content-page/app-innovation-radar-list/div[2]/app-show-more/div"):
        #espera "ver mais" ser clicável
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/app-root/mat-drawer-container/mat-drawer-content/div/div/div/app-innovation-radar-page/app-container/div/div/app-paginated-content/div/content-page/app-innovation-radar-list/div[2]/app-show-more/div")))
        #clica em ver mais
        click(driver, "/html/body/app-root/mat-drawer-container/mat-drawer-content/div/div/div/app-innovation-radar-page/app-container/div/div/app-paginated-content/div/content-page/app-innovation-radar-list/div[2]/app-show-more/div")

#armazenar primeira aba
original_window = driver.current_window_handle
 

#mapeando todos botões "veja os detalhes"
buttons =  driver.find_elements(By.CSS_SELECTOR, "[_nghost-serverApp-c111]   span[_ngcontent-serverApp-c111]   button[_ngcontent-serverApp-c111]")
print(len(buttons))


for button in buttons:
#abre nova aba
    ActionChains(driver)\
            .key_down(Keys.LEFT_CONTROL)\
            .click(button)\
            .key_up(Keys.LEFT_CONTROL)\
            .perform()

    wait = WebDriverWait(driver, 10)
    wait.until(EC.number_of_windows_to_be(2))

#troca para nova aba
    for window_handle in driver.window_handles:
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                break

    #pegar cabeçalho
    cabecalho = obter_elemento(driver, "/html/body/app-root/mat-drawer-container/mat-drawer-content/div/div/div/app-innovation-radar-details-page/app-container/div/app-content-area/div[1]/article" )
    #pegar conteúdo Texto
    conteudo = obter_elemento(driver,"/html/body/app-root/mat-drawer-container/mat-drawer-content/div/div/div/app-innovation-radar-details-page/app-container/div/app-content-area/div[1]/app-content-editor")
    #pegar contato
    contato = obter_elemento(driver, "/html/body/app-root/mat-drawer-container/mat-drawer-content/div/div/div/app-innovation-radar-details-page/app-container/div/app-content-area/div[1]/div[3]")
    titulo = obter_elemento(driver, "/html/body/app-root/mat-drawer-container/mat-drawer-content/div/div/div/app-innovation-radar-details-page/app-container/div/app-content-area/div[1]/article/section[1]/app-title-h1/h1")

    #tratando título das matérias que possuem / no nome 
    nomearquivo = (f"{titulo.text.replace('/', '_')}.txt")
    #criando arquivo
    f = open(nomearquivo, "x")
    f.write(cabecalho.text+"\n")
    f.write(conteudo.text+"\n")
    f.write(contato.text+"\n")
    f.close()

    #fechando janela
    driver.close()

    #voltando para a janela original
    driver.switch_to.window(original_window)


110
